# Creating and Populating a Database


**Content:** Create your first database, create tables and associated data for example used afterwards, learn about various data types, how to create tables using them, all towards MySQL features and syntax 

### Getting Ready

The following step only aquires the relevant information to start using my locally installed `mysql` server and makes everything ready to load the Sakila example database. 

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Retrieve variables
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_PASSWORD')
HOST = os.getenv('MYSQL_HOST', 'localhost')  # default fallback
PORT = os.getenv('MYSQL_PORT', 3306)
DATABASE = os.getenv('MYSQL_DATABASE')

To be ready to work with SQL in python, we load a few more libraries

In [2]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
import pymysql

...and quickly test if everything is working:

In [3]:
# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

# Test connection with a simple query
query = "SELECT * FROM film LIMIT 5;"
df = pd.read_sql(query, engine)

# Display results
df

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


## Creating a MySQL Database

### Using the mysql Command-Line Tool

To interact with the database, you execute `mysql` in your shell. Logging in using the root account is done by
```
    mysql -u root -p;
```
To see all available databases, you can use the following command:
```
    mysql> show databases;
```
In the following, I will additionally start showing how this can be done using python. First, we create a SQLAlchemy engine to define where we want to be (either in the main directory like below, or already in a specific database). We then set and execute the query and display the results.

In [4]:
# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}')
# Show all databases
query = "show databases"
df = pd.read_sql(query, engine)

# Display results
df

,Database
0,information_schema
1,mysql
2,performance_schema
3,sakila
4,sys


To use the Sakila database, we can specify the database via the `use` command with `use sakila;`. Likewise, we can directly access the Sakila database via
```
    mysql -u root -p sakila;
```
Now we are able to issue SQL statements and view the results. For example to know the current data and time, you could issue the following query:
```
    mysql> SELECT now();
```
In this notebook these steps are done by

In [5]:
# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

# Access current time
query = "SELECT now();"
df = pd.read_sql(query, engine)

# Display results
df

,now()
0,2025-07-01 11:49:56


If you access MySQL through a shell, and use the command lines, it will also display the number of rows that were returned, along with the execution time. If you want to display that as well, you can do so with a simple modified query function:

In [6]:
import time
# Function to run and time SQL query
def run_query(query, engine):
    start_time = time.time()
    df = pd.read_sql(query, engine)
    duration = time.time() - start_time
    print(f"✅ {len(df)} row(s) in set ({duration:.3f} sec)")
    return df

query = "SELECT now();"
df = run_query(query, engine)
# Display results
df

✅ 1 row(s) in set (0.002 sec)


,now()
0,2025-07-01 11:49:56


In some database servers, you need a `from` clause that names at least one table. In this case, the previous query can be written as
```
    mysql> SELECT now()
              FROM dual;
```

In [7]:
# Access current time with a FROM clause
query ="""
SELECT now() 
FROM dual;
"""
df = pd.read_sql(query, engine)

# Display results
df

,now()
0,2025-07-01 11:49:56


Once, we are done with everything, we should type `quit` or `exit` in the `mysql` command-line tool to return to the terminal/shell. In python when using `SQLAlchemy` there is no such equivalent as `mysql>exit`. Connections are typically pooled and managed automatically. What can be done optionally is to execute `engine.dispose()`, especially in long-running notebooks. But it's not neccessary in small projects like this.

In [8]:
#engine.dispose()  # Closes all connections in the pool

## MySQL Data Types

### Character Data

* *fixed-length strings*: right-padded (fill characters added to the right) with spaces and always consume the same number of bytes. For example abbreviations can be stored with fixed-length `char`. The definition is the following (for fixed 20 character length)
```
    char(20) /* fixed-length */
```
* *variable-length strings*: not right-padded with spaces and don't always consume the same number of bytes, given by (with maximal 20 characters)
```
    varchar(20) /* variable-length */
```

When defining a character column, the maximum size of any string to be stored needs to be specified. The maximum lenght for `char` is 255 bytes, whereas `varchar` can be up to 65,535. For longer texts (emails, XML documents) other text types (`mediumtext`, `longtext`) will be used.

#### Character Sets

Depending on the alphabet used, each character can be stored in a single byte (e.g. Latin alphabet) or multiple bytes are required (multibyte character sets) as for example Japanese or Korean. All supported character sets by MySQL can be shown with: 

In [9]:
# Access current time with a FROM clause
query ="SHOW CHARACTER SET;"
df = pd.read_sql(query, engine)

# Display results
df

,Charset,Description,Default collation,Maxlen
0,armscii8,ARMSCII-8 Armenian,armscii8_general_ci,1
1,ascii,US ASCII,ascii_general_ci,1
2,big5,Big5 Traditional Chinese,big5_chinese_ci,2
3,binary,Binary pseudo charset,binary,1
4,cp1250,Windows Central European,cp1250_general_ci,1
5,cp1251,Windows Cyrillic,cp1251_general_ci,1
6,cp1256,Windows Arabic,cp1256_general_ci,1
7,cp1257,Windows Baltic,cp1257_general_ci,1
8,cp850,DOS West European,cp850_general_ci,1
9,cp852,DOS Central European,cp852_general_ci,1


All character sets with `Maxlen`>1 are multibyte character sets. The default character set is `utf8mb4`. However, you can choose a different character set for each character column in your database, and store different character sets within the same table. A character set other than the default is chosen as:
```
    varchar(20) character set latin1
```
To set the default character set for the entire database, you can do
```
    create database european_sales character set latin1;
```
For more information on character sets, check out  "Unicode Explained: Internationalize Documents, Programs, and Web Sites (O'Reilly) - Jukka Korpela".

#### Text Data

Longer texts (exceeding 64KB limit for `varchar`), you need one of the following text types:

| Text Type        | Maximum number of bytes     |
|------------------|-----------------------------|
| `tinytext`       | 255                         |
| `text`           | 65,535                      |
| `mediumtext`     | 16,777,215                  |
| `longtext`       | 4,294,967,295               |

Be aware of the following:
* the data will be truncated if exceeding the maximum size for that type
* trailing spaces (spaces at the end) will not be removed when data is loaded
* if `text` is used for sorting or grouping, only the first 1,024 bytes are used (but can be increased)
* different text types are unique to the specific SQL server
* since `varchar` allows up to 65,535 bytes, there is no need for `tinytext` or `text`. 

Rule of thumb:
* for `notes` about customers `varchar` will probably be adequate.
* for storing documents `mediumtext` or `longtext` are better.

### Numeric Data

There are several different numeric data types depending on the way in which numbers are used. Examples include:

* *A column indicating whether a customer order has been shipped*: $\to $ ``Boolean`` contains ``0`` for ``false`` and a ``1`` for ``true``.

* *A system-generated primary key for a transaction table*: $\to$ This data would start at 1 and increase incrementally by one up to large numbers

* *An item number for a customer's electronic shopping basket*: $\to$ positive whole numbers between 1 and something around 200 maybe.

* *Positional data for a circuit board drill machine*: $\to$ high-precision data requires accuracy to several decimal points

MySQL has several different numeric data types to handle the data, with the most commonly used types being **whole numbers**, or *integers*. Additionally the data can be *unsigned* ($\geq 0$). The MySQL whole number integer data types are the following:

| Type        |           signed range          | unsigned range     |
|-------------|---------------------------------|--------------------|
| `tinyint`   | -128 to 127                     | 0 to 255           |
| `smallint`  | -32,768 to 32,767               | 0 to 65,535        |
| `mediumint` | -8,388,608 to 8,388,607         | 0 to 16,777,215    |
| `int`       | -2,147,483,648 to 2,147,483,647 | 0 to 4,294,967,295 |
| `bigint`    | -$2^{63}$ to $2^{63}-1$         | 0 to $2^{64}-1$    |

You should always try to choose a type that will be large enough to store the biggest number needed while not wasting storage space. For **floating-point numbers**, the following numeric types are available:

| Type        |           numeric range          |
|-------------|----------------------------------|
| `float(p,s)`   | -3.402823466 $\cdot10^{38}$ to -1.175494351 $\cdot 10^{-38}$ <br> and 1.175494351 $\cdot 10^{-38}$ to -3.402823466 $\cdot 10^{38}$                  |
| `double(p,s)` | -1.7976931348623157 $\cdot 10^{308}$ to -2.2250738585072014 $\cdot 10^{-308}$ <br> and 2.2250738585072014 $\cdot 10^{-308}$ to 1.7976931348623157 $\cdot 10^{308}$|

For floating-point types, you can specifiy a *precision* ( ``p``: total number of allowable digits both to the left + to the right of the decimal point) and a *scale* (``s``: the number of allowable digits to the right of the decimal point). Example: ``float(4,2)`` would result in 17.8675 $\to$ 17.87, and 178.375 would give an error since even after rounding to two decimals, the number has more than the allowed number of digits.

### Temporal Data

Temporal data deals with information about dates and/or times. Some examples for *temporal* types of data include:

* Expected date for an event to happen, such as shipping a customer's order
* Date that the customer's order was shipped
* Date and time that a user modified a particular row in a table
* Employee's birth date
* Year corresponding to a fact table, for example yearly sales in a warehouse
* Elapsed time to complete an assembly line

MySQL can handle all these situations with the following temporal data types:

| Type        |  Default format       | Allowable values                                             |
|-------------|-----------------------|--------------------------------------------------------------|
| `date`      | `YYYY-MM-DD`          | `1000-01-01` to `9999-12-31`                                 |
| `datetime`  | `YYYY-MM-DD HH:MI:SS` | `1000-01-01 00:00:00.000000` to `9999-12-31 23:59:59.999999` |
| `timestamp` | `YYYY-MM-DD HH:MI:SS` | `1970-01-01 00:00:00.000000` to `2038-01-18 22:14:07.999999` |
| `year`      | `YYYY`                | `1901` to `2155`                                             |
| `time`      | `HHH:MI:SS`           | `-838:59:59.000000` to `838:59:59.000000`                    |

For the data types `datatime`, `timestamp`, and `time`, you can, for example, specify `datetime(2)` which allows your time values to include only hundredths of a second instead of the maximum of up to 6 decimal places (microseconds). Note that MySQL only handles dates starting in 1000 AD. Keep this in mind when storing historical dates. The date format components are mostly self-explaining. The only noticable component is `HH` vs `HHH` where `HHH` defines elapsed hours. So you could refer to hours in the past with respect to a certain event by giving it a negative value. 

Now you can ask yourself in which way you would implement the above given examples:

* Expected date for an event to happen, such as shipping a customer's order $\to$ `date` probably sufficient since hour can often not be determined for a shipping for example
* Date that the customer's order was shipped $\to$ `datetime` to track the time of the order and not only the date
* Date and time that a user modified a particular row in a table $\to$ `timestamp` column will automatically be populated with the current date/time by the MySQL server
* Employee's birth date $\to$ `date` since precision down to seconds not needed
* Year corresponding to a fact table, for example yearly sales in a warehouse $\to$ `year`
* Elapsed time to complete an assembly line $\to$ `time` since it is unnecessary to also track the date if you're only interested in the time duration.

## Table Creation

### Step 1: Design

Brainstorm about twhat information would be helpful. For example for a person, we might want to describe the person by:

* Name
* Eye color
* Birth date
* Address
* Favorite foods

As a next step, we assign column names and data types to the `person` table:

| Column           |  Type          | Allowable values |
|------------------|----------------|------------------|
| `name`           | `varchar(40)`  |                  |
| `eye_color`      | `char(2)`      | `BL, BR, GR`     |
| `birth_date`     | `date`         |                  |
| `address`        | `varchar(100)` |                  |
| `favorite_foods` | `varchar(200)` |                  |

The `name`, `address`, and `favorite_foods` columns are of type `varchar` and allow for free-form data entry. The `eye_color` column allows two characters that should equal only `BR, BL`, or `GR`. Since no time component is needed, the `birth_date` column is of type `date`.

### Step 2: Refinement

The following issues arrise if we look at the columns as they are for now:

* `name`, and `address` are a compound objects consisting of first and last name, and street, city, state/province, country, postal code, respectively (no normalization!).
* no uniqueness guaranteed since multiple people can have the same name, eye color, birth date
* `favorite_foods` can contain zero, one, or more independent items. It is better to create a separate table for this data that includes a foreign key to the `person` table.

A refined version of the `person` table now looks like this:

| Column           |  Type          | Allowable values |
|------------------|----------------|------------------|
| `person_id`           | `smallint (unsigned)`  |                  |
| `first_name`           | `varchar(20)`  |                  |
| `last_name`           | `varchar(20)`  |                  |
| `eye_color`      | `char(2)`      | `BL, BR, GR`     |
| `birth_date`     | `date`         |                  |
| `street`        | `varchar(30)` |                  |
| `city`        | `varchar(20)` |                  |
| `state`        | `varchar(20)` |                  |
| `country`        | `varchar(20)` |                  |
| `postal_code`        | `varchar(20)` |                  |

and for the `favorite_food` table, we have:


| Column           |  Type                  |
|------------------|------------------------|
| `person_id`      | `smallint (unsigned)`  |
| `food`           | `varchar(20)`          |

When should we stop refining the structure? You might want people to choose their favorite foods from a list of options, for example `pasta` instead of everyone entering types of pasta like `spaghetti`, `tagliatelle`, ... . In this case you should create a `food` table with `food_id`, and `food_name` columns, and then change the `favorite_food` table to contain a foreign key to the `food` table. 

### Step 3: Building SQL Schema Statements

Now let's generate the SQL statements to create the tables in the database. With the following statement, we will run the `create table` statement:

In [13]:
from sqlalchemy import text

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

# Create the `person` table 
query ="""
CREATE TABLE person
    (person_id SMALLINT UNSIGNED,
    fname VARCHAR(20),
    lname VARCHAR(20),
    eye_color ENUM('BR','BL','GR'),
    birth_date DATE,
    street VARCHAR(30),
    city VARCHAR(20),
    state VARCHAR(20),
    country VARCHAR(20),
    postal_code VARCHAR(20),
    CONSTRAINT pk_person PRIMARY KEY (person_id)
    );
"""

# Execute DDL query using SQLAlchemy engine
with engine.connect() as connection:
    connection.execute(text(query))
    print("✅ Table `person` created successfully.")

OperationalError: (pymysql.err.OperationalError) (1050, "Table 'person' already exists")
[SQL: 
CREATE TABLE person
    (person_id SMALLINT UNSIGNED,
    fname VARCHAR(20),
    lname VARCHAR(20),
    eye_color ENUM('BR','BL','GR'),
    birth_date DATE,
    street VARCHAR(30),
    city VARCHAR(20),
    state VARCHAR(20),
    country VARCHAR(20),
    postal_code VARCHAR(20),
    CONSTRAINT pk_person PRIMARY KEY (person_id)
    );
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Apparently, the `person` table already exists. All lines above are self-explanatory, except of:
1. `eye_color ENUM('BR','BL','GR')` which is adding the constraint that only these values are allowed. Another possibilty would be in most servers `eye_color CHAR(2) CHECK (eye_color IN ('BR','BL','GR'))`. This is a *check constraint*.
2. `CONSTRAINT pk_person PRIMARY KEY (person_id)` which tells the database what column or columns will serve as the primary key for the table. This is a *primary key constraint*.

We can now check if the table does, in fact, exist:

In [14]:
df = pd.read_sql("SHOW TABLES LIKE 'person';", engine)
df

,Tables_in_sakila (person)
0,person


In [15]:
df = pd.read_sql("DESCRIBE person;", engine)
df

,Field,Type,Null,Key,Default,Extra
0,person_id,smallint unsigned,NO,PRI,None,
1,fname,varchar(20),YES,,None,
2,lname,varchar(20),YES,,None,
3,eye_color,"enum('BR','BL','GR')",YES,,None,
4,birth_date,date,YES,,None,
5,street,varchar(30),YES,,None,
6,city,varchar(20),YES,,None,
7,state,varchar(20),YES,,None,
8,country,varchar(20),YES,,None,
9,postal_code,varchar(20),YES,,None,


In the command-line this would be

```
    SHOW TABLES LIKE 'person';
```
and
```
    mysql> desc person;
```

Let's quickly discuss the output of `mysql> desc person;`. Column 3 shows whether a particular colun can be omitted when data is inserted into the table. Column 4 shows whether a column takes part in any keys (primary or foreign). Column 5 tells you if a default value will be inserted if nothing else is specified. Column 6 collects any other pertinent information that might apply to a column. `NULL` or `None` is used for cases where a value cannot be supplied because it is: not applicable, unknown, or an empty set. 

We can now continue creating the `favorite_food` table:

In [16]:
# Create the `person` table 
query ="""
CREATE TABLE favorite_food
    (person_id SMALLINT UNSIGNED,
    food VARCHAR(20),
    CONSTRAINT pk_favorite_food PRIMARY KEY (person_id, food),
    CONSTRAINT fk_fav_food_person_id FOREIGN KEY (person_id)
    REFERENCES person (person_id)
    );
"""

# Execute DDL query using SQLAlchemy engine
with engine.connect() as connection:
    connection.execute(text(query))
    print("✅ Table `person` created successfully.")  

✅ Table `person` created successfully.


Compared to the first table, we have two major differences:
1. It takes more than just the `person_id` column as a primary key to guarantee uniqueness in the table: `person_id` and `food`.
2. It contains a *foreign key constraint*. This constraint ensures that only `person_id` values that have been defined in the `person` table before can be used here. Foreign keys can be also added later with `alter table`.

Let's see how the table looks like now:

In [17]:
df = pd.read_sql("DESCRIBE favorite_food;", engine)
df

,Field,Type,Null,Key,Default,Extra
0,person_id,smallint unsigned,NO,PRI,None,
1,food,varchar(20),NO,PRI,None,


## Populating and Modifying Tables

Now that the tables are created, they can be populated with data.